Consider the following optimization problem: we wish to find a polynomial $p(x)=a+bx+cx^2+...$ of degree $n$ such that $p'(0)$ is maximized on $|x| \leq 1$, subject to $|p(x)| \leq 1$.

Now, note that $p'(0)=b$. The problem is that our function and our constraints are polynomial, not convex, which is a problem for our solvers. However, we can get around this problem as follows: randomly sample a number of points $x_i$, and evaluate our constraints at those points; the resulting set of constraints are used to formulate and solve a linear program, which approximates the exact solution well.

In [23]:
import numpy as np
import cvxpy as cp
np.random.seed(1234)
np.set_printoptions(precision=2, suppress=True)

def pol_calc(c, x): # calculation of polynomial at points x
    return sum(c[i] * (x**i) for i in range(c.size))
    
def solve_degree_n(n, n_sample=1e4, verbose=True):
    C = cp.Variable((n+1))
    x = np.random.uniform(-1,1,n_sample)
    pol_x = pol_calc(C, x) # evaluate polynomial at x
    constraints = [px_i>=-1 for px_i in pol_x] + [px_i<=1 for px_i in pol_x]
    
    problem = cp.Problem(cp.Maximize(C[1]), constraints)
    problem.solve(solver=cp.GLPK)
    if verbose:
        print(problem.status)
        print(problem.value)
    
    return C.value

In [22]:
for n in range(1,9):
    sol=solve_degree_n(n, 5000, verbose=False)
    print("For degree: " + str(n) + ":")
    print(sol)

For degree: 1:
[0. 1.]
For degree: 2:
[ 0.5  1.  -0.5]
For degree: 3:
[-0.  3.  0. -4.]
For degree: 4:
[ 0.  3. -0. -4. -0.]
For degree: 5:
[ -0.     5.     0.01 -20.01  -0.01  16.01]
For degree: 6:
[  0.     5.    -0.   -20.     0.01  16.    -0.01]
For degree: 7:
[  0.     7.    -0.04 -56.08   0.15 112.27  -0.12 -64.22]
For degree: 8:
[  0.     7.    -0.01 -56.02   0.04 112.07  -0.05 -64.06   0.02]


With the random sampling we have obtained a good approximation of the optimal solution, and if we round off the small decimals we can start to discern some patterns in the coefficients. We see that for degree $2k$ the coefficients are the same as for degree $2k-1$, with the exception of degree 2. Looking closer, we might notice that the polynomial of degree $2k-1$ is the [Chebyshev polynomials of the first kind](https://en.wikipedia.org/wiki/Chebyshev_polynomials) of degree $2k-1$ (up to a change in sign, which might flip to maximize $b$). Subsequently, we can use the following fact about Chebyshev polynomials of the first kind: in general 

$|p'(0)| \leq k$.